In [1]:
import numpy as np
import sklearn.metrics as metrics
import tensorflow as tf
import os, os.path
import pandas as pd
import math

import wide_residual_network_fix_v2 as wrn
from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

from keras import backend as K
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
name = 'KaggleV2.0.3.1 - WRN-16-2-SGD_fix'

if not os.path.exists(name):
    os.makedirs(name)

In [3]:
# GPU Memory limitations
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# set_session(tf.Session(config=config))

In [4]:
lr_schedule = [60, 120, 160] # epoch_step
def schedule(epoch_idx):
    if (epoch_idx + 1) < lr_schedule[0]:
        return 0.1
    elif (epoch_idx + 1) < lr_schedule[1]:
        return 0.02 # lr_decay_ratio = 0.2
    elif (epoch_idx + 1) < lr_schedule[2]:
        return 0.004
    return 0.0008

In [5]:
batch_size = 128
nb_epoch = 200
img_rows, img_cols = 32, 32

(trainX, trainY), (testX, testY) = cifar10.load_data()

trainX = trainX.astype('float32')
trainX /= 255.0
testX = testX.astype('float32')
testX /= 255.0

tempY = testY
trainY = kutils.to_categorical(trainY)
testY = kutils.to_categorical(testY)

generator = ImageDataGenerator(featurewise_center=True,
                               featurewise_std_normalization=True,
                               rotation_range=10,
                               width_shift_range=5./32,
                               height_shift_range=5./32,
                               horizontal_flip=True)

generator.fit(trainX, seed=0, augment=True)

test_generator = ImageDataGenerator(featurewise_center=True,
                                    featurewise_std_normalization=True)

test_generator.fit(testX, seed=0, augment=True)

init_shape = (3, 32, 32) if 0 == 'th' else (32, 32, 3)


In [10]:
# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
model = wrn.create_wide_residual_network(init_shape, nb_classes=10, N=2, k=2, dropout=0.0)

model.summary()
plot_model(model, "WRN-16-2.png", show_shapes=True)

opt = SGD(lr=0.01, momentum=0.9, nesterov=True, decay=0.0005)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])
print("Finished compiling")
print("Allocating GPU memory")

model.load_weights(name + '/WRN-16-2 Weights-hand_saved_FULL.h5')
print("Model loaded.")

Wide Residual Network-10-2 created.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
zero_padding2d_15 (ZeroPadding2D (None, 34, 34, 3)     0           input_3[0][0]                    
____________________________________________________________________________________________________
conv2d_33 (Conv2D)               (None, 34, 34, 16)    448         zero_padding2d_15[0][0]          
____________________________________________________________________________________________________
batch_normalization_25 (BatchNor (None, 34, 34, 16)    64          conv2d_33[0][0]                  
_______________________________________________________

Finished compiling
Allocating GPU memory
Model loaded.


In [11]:
# Callbacks
csv_logger = callbacks.CSVLogger(name + '/log.csv', append=True, separator=';')
save_c = callbacks.ModelCheckpoint(name + '/WRN-16-2 Weights.h5', monitor="val_acc", save_best_only=True)
lrs = LearningRateScheduler(schedule=schedule)

In [12]:
model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size + 1, nb_epoch=nb_epoch,
                   callbacks=[csv_logger, save_c, lrs],
                   validation_data=test_generator.flow(testX, testY, batch_size=batch_size),
                   validation_steps=testX.shape[0] // batch_size + 1,)

scores = model.evaluate_generator(test_generator.flow(testX, testY, nb_epoch), testX.shape[0] // batch_size + 1)
print("Accuracy = %f" % (100 * scores[1]))

c:\users\intel\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
c:\users\intel\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=200, callbacks=[<keras.ca..., validation_steps=79, steps_per_epoch=391)`
  after removing the cwd from sys.path.


Epoch 1/200
391/391 [==============================] - 46s - loss: 2.5926 - acc: 0.2842 - val_loss: 2.4856 - val_acc: 0.2628
Epoch 2/200
391/391 [==============================] - 43s - loss: 1.8773 - acc: 0.4471 - val_loss: 2.3625 - val_acc: 0.2716
Epoch 3/200
391/391 [==============================] - 43s - loss: 1.5168 - acc: 0.5570 - val_loss: 1.7780 - val_acc: 0.4609
Epoch 4/200
391/391 [==============================] - 43s - loss: 1.2507 - acc: 0.6459 - val_loss: 1.6604 - val_acc: 0.5186
Epoch 5/200
391/391 [==============================] - 43s - loss: 1.0777 - acc: 0.7045 - val_loss: 1.2659 - val_acc: 0.6578
Epoch 6/200
391/391 [==============================] - 43s - loss: 0.9831 - acc: 0.7408 - val_loss: 1.1722 - val_acc: 0.6714
Epoch 7/200
391/391 [==============================] - 43s - loss: 0.9199 - acc: 0.7621 - val_loss: 1.0715 - val_acc: 0.7074
Epoch 8/200
391/391 [==============================] - 43s - loss: 0.8746 - acc: 0.7766 - val_loss: 1.1173 - val_acc: 0.7071


391/391 [==============================] - 42s - loss: 0.2945 - acc: 0.9585 - val_loss: 0.5157 - val_acc: 0.8878
Epoch 67/200
391/391 [==============================] - 42s - loss: 0.2895 - acc: 0.9604 - val_loss: 0.5276 - val_acc: 0.8890
Epoch 68/200
391/391 [==============================] - 42s - loss: 0.2869 - acc: 0.9610 - val_loss: 0.5376 - val_acc: 0.8852
Epoch 69/200
391/391 [==============================] - 42s - loss: 0.2801 - acc: 0.9628 - val_loss: 0.5331 - val_acc: 0.8885
Epoch 70/200
391/391 [==============================] - 42s - loss: 0.2817 - acc: 0.9619 - val_loss: 0.5333 - val_acc: 0.8872
Epoch 71/200
391/391 [==============================] - 42s - loss: 0.2766 - acc: 0.9627 - val_loss: 0.5506 - val_acc: 0.8832
Epoch 72/200
391/391 [==============================] - 42s - loss: 0.2736 - acc: 0.9634 - val_loss: 0.5247 - val_acc: 0.8909
Epoch 73/200
391/391 [==============================] - 42s - loss: 0.2715 - acc: 0.9639 - val_loss: 0.5568 - val_acc: 0.8801
Epoch

391/391 [==============================] - 42s - loss: 0.1796 - acc: 0.9837 - val_loss: 0.4945 - val_acc: 0.8982
Epoch 195/200
391/391 [==============================] - 42s - loss: 0.1799 - acc: 0.9839 - val_loss: 0.5182 - val_acc: 0.8930
Epoch 196/200
391/391 [==============================] - 42s - loss: 0.1799 - acc: 0.9846 - val_loss: 0.5129 - val_acc: 0.8939
Epoch 197/200
391/391 [==============================] - 42s - loss: 0.1795 - acc: 0.9841 - val_loss: 0.5073 - val_acc: 0.8961
Epoch 198/200
391/391 [==============================] - 42s - loss: 0.1795 - acc: 0.9844 - val_loss: 0.5126 - val_acc: 0.8954
Epoch 199/200
391/391 [==============================] - 42s - loss: 0.1789 - acc: 0.9843 - val_loss: 0.5080 - val_acc: 0.8952
Epoch 200/200
391/391 [==============================] - 42s - loss: 0.1792 - acc: 0.9838 - val_loss: 0.5176 - val_acc: 0.8942
Accuracy = 89.765824


In [49]:
def submitFile(modelFile, saveFile, model):
    model.load_weights(modelFile)
    test_x = np.load('All images.npy')
    test_x = test_x[1]/255.
    bs = 100
    steps = math.ceil(int(test_x.shape[0])/bs)
    print ("Steps - " + str(steps))
    pred = model.predict_generator(test_generator.flow(test_x, batch_size=bs, shuffle=False), steps=steps)
    y_pred = np.argmax(pred, axis=1)
    labels = labelsToStrings(y_pred)
    labels.to_csv(saveFile)

In [50]:
def labelsToStrings(data):
    ret = []
    for label in data:
        temp = ''
        
        if(label == 0): temp = 'airplane'
        if(label == 1): temp = 'automobile'
        if(label == 2): temp = 'bird'
        if(label == 3): temp = 'cat'
        if(label == 4): temp = 'deer'
        if(label == 5): temp = 'dog'
        if(label == 6): temp = 'frog'
        if(label == 7): temp = 'horse'
        if(label == 8): temp = 'ship'
        if(label == 9): temp = 'truck'
        
        ret.append(temp)
    
    label_id = np.arange(data.shape[0]) + 1
    s = pd.Series(ret, index=label_id)
    s = s.to_frame('label')
    s.index.name='id'
    
    return s

In [51]:
# submitFile(modelFile=name + '/WRN-16-2 Weights.h5', saveFile='cifar10_submit_V2.0.2.csv', model=model)

Steps - 3000


In [8]:
# model.save_weights(name + '/WRN-28-8 Weights-hand_saved.h5')

In [18]:
model.save(name + '/WRN-16-2 Weights-hand_saved_FULL.h5')

In [1]:
import pydot

In [7]:
import graphviz

In [2]:
import wide_residual_network_fix_v2 as wrn

Using TensorFlow backend.
